<a href="https://colab.research.google.com/github/veronica1908/MARKETING_E2_A_III/blob/main/Preprocesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Instalar la extensión ipython-sql
!pip install -q ipython-sql

# Cargar la extensión
%load_ext sql

# Instalar bibliotecas necesarias
!pip install sqlalchemy
!pip install mysql-connector-python

# Importar bibliotecas
import sqlite3
from sqlalchemy import create_engine
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 59.4 MB/s eta 0:00:00


In [6]:
# Conectarse a la base de datos
conn = sqlite3.connect('db_movies')
# Crear un cursor
cur = conn.cursor()

In [7]:
# Obtener información sobre las columnas de la tabla 'ratings'
cur.execute("PRAGMA table_info(ratings)")
columns_ratings = cur.fetchall()

# Imprimir el nombre de la columna y su tipo de datos para la tabla 'ratings'
print("Tipos de datos de las columnas en la tabla 'ratings':")
for column in columns_ratings:
    print(column[1] + ": " + column[2])

# Obtener información sobre las columnas de la tabla 'movies'
cur.execute("PRAGMA table_info(movies)")
columns_movies = cur.fetchall()

# Imprimir el nombre de la columna y su tipo de datos para la tabla 'movies'
print("\nTipos de datos de las columnas en la tabla 'movies':")
for column in columns_movies:
    print(column[1] + ": " + column[2])

Tipos de datos de las columnas en la tabla 'ratings':
userId: INTEGER
movieId: INTEGER
rating: REAL
timestamp: INTEGER

Tipos de datos de las columnas en la tabla 'movies':
movieId: INTEGER
title: TEXT
genres: TEXT


In [8]:
#Extraer el año de lanzamiento de la película en la tabla movies, el cual está incluido en el título de película
#Agregar una nueva columna para almacenar el año

conn = sqlite3.connect('db_movies')
cursor = conn.cursor()

# Agregar una nueva columna para almacenar el año
cursor.execute("ALTER TABLE movies ADD COLUMN year INTEGER")

# Actualizar la nueva columna con el año extraído del título de la película
cursor.execute("""
    UPDATE movies
    SET year = CAST(SUBSTR(title, -5, 4) AS INTEGER)
""")

# Confirmar los cambios y cerrar la conexión
conn.commit()
conn.close()

In [9]:
#Verificamos las columnas de la tabla 'movies'

conn = sqlite3.connect('db_movies')
cursor = conn.cursor()

# Obtener información sobre las columnas de la tabla 'movies'
cursor.execute("PRAGMA table_info(movies)")
columns_FR = cursor.fetchall()

# Imprimir el nombre de la columna y su tipo de datos
for column in columns_FR:
    print(column[1], "-", column[2])

conn.close()

movieId - INTEGER
title - TEXT
genres - TEXT
year - INTEGER


In [10]:
# verificamos el contenido de las primeras filas de la tabla movies
conn = sqlite3.connect('db_movies')

query = "SELECT * FROM movies LIMIT 5"

# Ejecutar la consulta y cargar los resultados en un DataFrame de Pandas
movies = pd.read_sql(query, conn)

# Mostrar las primeras filas del DataFrame
print(movies)

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  year  
0  Adventure|Animation|Children|Comedy|Fantasy  1995  
1                   Adventure|Children|Fantasy  1995  
2                               Comedy|Romance  1995  
3                         Comedy|Drama|Romance  1995  
4                                       Comedy  1995  


In [11]:
# verificamos el contenido de las primeras filas de la tabla ratings
conn = sqlite3.connect('db_movies')

query = "SELECT * FROM ratings LIMIT 5"

# Ejecutar la consulta y cargar los resultados en un DataFrame de Pandas
ratings = pd.read_sql(query, conn)

# Mostrar las primeras filas del DataFrame
print(ratings)

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [12]:
# Crear tabla con usuarios con más de 50 películas vistas y menos de 1000
query = """
    DROP TABLE IF EXISTS usuarios_sel;
    CREATE TABLE usuarios_sel AS
    SELECT "userId" AS user_id, COUNT(*) AS cnt_rat
    FROM ratings
    GROUP BY "userId"
    HAVING cnt_rat > 50 AND cnt_rat <= 1000
    ORDER BY cnt_rat DESC;
"""

# Ejecutar la consulta
conn.executescript(query)
# Cerrar la conexión
conn.close()

In [13]:
# Crear tabla con películas que han sido calificadas por más de 50 usuarios
conn = sqlite3.connect('db_movies')

query = """
DROP TABLE IF EXISTS movies_sel;

CREATE TABLE movies_sel AS
SELECT "movieId",
       COUNT(*) AS cnt_rat
FROM ratings
GROUP BY "movieId"
HAVING cnt_rat > 50
ORDER BY cnt_rat DESC;
"""

# Ejecutar la consulta
conn.executescript(query)

conn.close()

In [14]:
# Crear tablas filtradas de calificaciones con usuarios y películas
##ratings_final
conn = sqlite3.connect('db_movies')

query = """
DROP TABLE IF EXISTS ratings_final;

CREATE TABLE ratings_final AS
SELECT a."userId" AS user_id,
       a."movieId" AS movie_id,
       a."rating" AS rating
FROM ratings a
INNER JOIN movies_sel b ON a.movieId = b.movieId
INNER JOIN usuarios_sel c ON a."userId" = c.user_id;
"""
conn.executescript(query)
conn.close()

In [15]:
##movies_final
# Establecer conexión a la base de datos
conn = sqlite3.connect('db_movies')

# Definir la consulta SQL para crear la tabla movies_final
query = """
DROP TABLE IF EXISTS movies_final;

CREATE TABLE movies_final AS
SELECT a.movieId AS movie_id,
       a.title AS movie_title,
       a.genres AS genres,
       a.year
FROM movies a
INNER JOIN movies_sel c ON a.movieId = c.movieId;
"""

# Ejecutar la consulta
conn.executescript(query)

# Cerrar la conexión a la base de datos
conn.close()

In [16]:
conn = sqlite3.connect('db_movies')
cursor = conn.cursor()

# Script SQL
script = """
DROP TABLE IF EXISTS full_ratings;

CREATE TABLE full_ratings AS
SELECT a.*,
       c.movie_title,
       c.genres,
       c. year
FROM ratings_final a
INNER JOIN movies_final c ON a.movie_id = c.movie_id;
"""

# Ejecutar el script SQL
conn.executescript(script)

# Confirmar los cambios y cerrar la conexión
conn.commit()
conn.close()

print("Tabla 'full_ratings' creada con éxito.")

conn.close()

Tabla 'full_ratings' creada con éxito.


In [17]:
# Consulta SQL para seleccionar las primeras filas de la tabla full_ratings
conn = sqlite3.connect('db_movies')

query = "SELECT * FROM full_ratings LIMIT 5"  # Cambia el número de filas según tus necesidades

# Ejecutar la consulta y cargar los resultados en un DataFrame de Pandas
full_ratings = pd.read_sql(query, conn)

# Mostrar las primeras filas del DataFrame
print(full_ratings)

   user_id  movie_id  rating                  movie_title  \
0        1         1     4.0             Toy Story (1995)   
1        1         3     4.0      Grumpier Old Men (1995)   
2        1         6     4.0                  Heat (1995)   
3        1        47     5.0  Seven (a.k.a. Se7en) (1995)   
4        1        50     5.0   Usual Suspects, The (1995)   

                                        genres  year  
0  Adventure|Animation|Children|Comedy|Fantasy  1995  
1                               Comedy|Romance  1995  
2                        Action|Crime|Thriller  1995  
3                             Mystery|Thriller  1995  
4                       Crime|Mystery|Thriller  1995  


# Exportar la tabla preprocesada en SQL

In [18]:
# Consulta SQL para seleccionar todos los datos de la tabla full_ratings
query = "SELECT * FROM full_ratings"

# Leer los datos en un DataFrame de pandas
df = pd.read_sql_query(query, conn)

# Guardar el DataFrame como un archivo CSV
df.to_csv('full_ratings.csv', index=False)

print("La tabla 'full_ratings' se ha exportado como full_ratings.csv")

# Cerrar la conexión
conn.close()

La tabla 'full_ratings' se ha exportado como full_ratings.csv


In [19]:
#Verificar tablas creadas en el script
# Conectarse a la base de datos
conn = sqlite3.connect('db_movies')

# Crear un cursor
cur = conn.cursor()

# Obtener una lista de tablas en la base de datos
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()

# Imprimir los nombres de las tablas para verificar la información
for table in tables:
    print(table[0])

ratings
movies
usuarios_sel
movies_sel
ratings_final
movies_final
full_ratings
